In [ ]:
%load_ext autoreload
%autoreload 2

%cd '..'

In [2]:
import logging

# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s",
#     handlers=[
#         logging.FileHandler("data/logs/preprocess_comments_notebook.log"),
#         logging.StreamHandler()
#     ]
# )

In [3]:
import json

import numpy as np
import polars as pl
from tqdm import tqdm
tqdm.pandas()

from load.utils import (
    load_users,
    load_subreddits,
    load_user_party,
    load_comments,
    save_df_as_parquet,
)
from preprocessing.utils import (
    tokenize_comment,
    calculate_user_party,
    save_event_comments,
    build_vocab,
    save_event_vocab,
)

from preprocessing.constants import (
    METADATA_DIR,
    ELECTIONS_REGEX,
    MIN_OCCURENCE_FOR_VOCAB,
)

In [4]:
YEAR = 2008
START_MONTH = 1
STOP_MONTH = 12

EVENT_NAME = f"us_elections_{YEAR}"

### User Affiliation

In [5]:
subreddits = load_subreddits()

logging.info(subreddits.dtypes)

In [6]:
# take into account network structure to find other partisan subreddits which are not labeled
dem_subreddits = set(
    json.load(
        open(
            f"{METADATA_DIR}/dem_subreddits_{YEAR}_unweighted.json",
        )
    )
)
rep_subreddits = set(
    json.load(
        open(
            f"{METADATA_DIR}/rep_subreddits_{YEAR}_unweighted.json",
        )
    )
)

partisan_subreddits = dem_subreddits | rep_subreddits


FileNotFoundError: [Errno 2] No such file or directory: '/dss/dsshome1/03/di93fup/polarization_reddit/data/metadata/dem_subreddits_2008_unweighted.json'

In [ ]:
# Filter partisan subreddits
# subreddits = subreddits[subreddits["party"].isin({"dem", "rep"})]
subreddits = subreddits.query("subreddit in @partisan_subreddits")

In [ ]:
logging.info(subreddits.groupby("party")["subreddit"].count())


In [ ]:
logging.info("Load comments...")

comments_pl = load_comments(
    year=YEAR,
    start_month=START_MONTH,
    stop_month=STOP_MONTH,
    engine="polars",
)

In [ ]:
partisan_comments_pl = comments_pl.filter(pl.col("subreddit").is_in(list(partisan_subreddits)))

In [ ]:
partisan_comments = partisan_comments_pl.to_pandas(
    # use_pyarrow_extension_array=True,
).astype(
    {
        "author": "string[pyarrow]",
        "body_cleaned": "string[pyarrow]",
        "created_utc": "int64[pyarrow]",
        "subreddit": "string[pyarrow]",
    }
)


In [ ]:
partisan_comments.dtypes

author          string[pyarrow]
body_cleaned    string[pyarrow]
created_utc      int64[pyarrow]
subreddit       string[pyarrow]
dtype: object

In [ ]:
# partisan_comments = comments.query("subreddit in @partisan_subreddits").copy()
# partisan_comments = comments.loc[t] # .copy()

logging.info(partisan_comments.shape)


In [ ]:
logging.info("Add party information to comments...")

partisan_comments["party"] = np.where(
    partisan_comments["subreddit"].isin(dem_subreddits), "dem", "rep"
)
partisan_comments["party"] = partisan_comments["party"].astype("string[pyarrow]")


In [ ]:
partisan_comments.dtypes

author          string[pyarrow]
body_cleaned    string[pyarrow]
created_utc      int64[pyarrow]
subreddit       string[pyarrow]
party           string[pyarrow]
dtype: object

In [ ]:
logging.info(partisan_comments.shape)

save_df_as_parquet(
    partisan_comments,
    target_file=f"partisan_comments_{YEAR}.parquet",
)


In [ ]:
user_party = partisan_comments.groupby(by="author").progress_apply(
    calculate_user_party,
)
user_party = user_party[user_party["score"] != 0].reset_index().copy()
user_party["party"] = user_party["party"].astype("string[pyarrow]")


100%|██████████| 33666/33666 [00:48<00:00, 688.11it/s]


In [ ]:
user_party.dtypes

author     string[pyarrow]
dem_cnt              int64
rep_cnt              int64
score                int64
party      string[pyarrow]
dtype: object

In [ ]:
logging.info(f"Nr of users: {len(user_party)}")

logging.info(user_party.groupby(by="party")["author"].count())

In [ ]:
save_df_as_parquet(
    data=user_party,
    target_file=f"user_party_{YEAR}.parquet",
)

logging.info(user_party.dtypes)

## Filter event comments

In [ ]:
# logging.info("Load user party")
# user_party = load_user_party(year=YEAR)

logging.info(user_party.dtypes)

In [ ]:
logging.info(partisan_comments.shape)

In [ ]:
users = load_users(engine="polars")

logging.info(users.dtypes)

In [ ]:
logging.info(users.shape)

In [ ]:
logging.info("Filter out bots & automoderators comments...")
partisan_comments = partisan_comments.merge(
    users,
    on="author",
    how="inner",
)

In [ ]:
logging.info(partisan_comments.dtypes)

In [ ]:
logging.info("Filtering event data based on keywords...")
event_comments = partisan_comments[
    partisan_comments["body_cleaned"].str.contains(
        ELECTIONS_REGEX[YEAR],
        regex=True,
    )
].copy()
logging.info("finished keyword filtering")

In [ ]:
logging.info(event_comments.shape)

In [ ]:
logging.info("save temp event comments")
save_event_comments(event_comments, f"temp_{EVENT_NAME}")


In [ ]:
del user_party
del users
del subreddits
del comments_pl
del partisan_comments

In [ ]:
logging.info(event_comments.dtypes)

## Tokenize and stem comments

In [ ]:
logging.info("Tokenizing comments...")
event_comments["tokens"] = event_comments["body_cleaned"].progress_apply(
    tokenize_comment,
).astype("string[pyarrow]")
logging.info("Finish tokenizing comments")

100%|██████████| 128307/128307 [02:54<00:00, 736.56it/s] 


In [ ]:
logging.info(event_comments.dtypes)

In [ ]:
logging.info(f"Nr of event comments: {len(event_comments)}")

In [ ]:
logging.info("saving event comments...")
save_event_comments(event_comments, EVENT_NAME)

## Build event vocabulary

In [ ]:
# Read event data
# events_comments = load_event_comments(
#     event_comments,
#     EVENT_NAME,
#     file_type="parquet",
# )

event_vocab = build_vocab(
    event_comments["tokens"],
    min_comment_freq=MIN_OCCURENCE_FOR_VOCAB,
)


In [ ]:
logging.info("Vocabulary length")
logging.info(len(event_vocab))

In [ ]:
logging.info("Saving event vocab")
save_event_vocab(event_vocab, EVENT_NAME)